In [1]:
from matplotlib.image import imread
from numpy import array,full,matmul,append,random
from os import listdir as ls
from regex import search
from math import exp

In [2]:
X_dataset = list()
Y_dataset = list()
for img in ls("./coil-20-proc/"):
	im = imread("./coil-20-proc/"+img)
	im = im.reshape(16384)
	im = append(im,-1)    
	X_dataset.append(im)
	result = search('obj(.*?)_', img)
	s=full(20,-1)
	s[int(result[1])-1]=1
	Y_dataset.append(s)
X_dataset = array(X_dataset)
Y_dataset = array(Y_dataset)

In [5]:
class MultilayerPerceptron:
    def __init__(self,hidden_layer,output_layer):
        self.hidden_layer = hidden_layer
        self.output_layer = output_layer
    
    
    def activation_func(self,weight,data):
        net = matmul(data.T,weight)
        return (1-exp(-net*.01))/(1+exp(-net*.01))
    
    def calculate_oh(self,desired,actual):
        return (desired-actual)*(1-actual**2)*.5
    
    def train(self,output_weight,hidden_weight,pattern,label,eta,emax):
        count = 0
        while True:
            error = 0
            for j in range(len(pattern)):
                hidden_output = []
                for i in range(self.hidden_layer):
                    
                    out = self.activation_func(hidden_weight[i],pattern[j])
                    hidden_output.append(out)
                hidden_output.append(-1)
                hidden_output = array(hidden_output).reshape(self.hidden_layer+1,1)
#                 print(hidden_output[i][0])
                delta_out = array([])
                for i in range(self.output_layer):
#                     print(output_weight[:,i].shape,hidden_output.shape)
                    out = self.activation_func(output_weight[:,i].reshape(len(hidden_output),1),hidden_output)
                    error = error + 0.5*pow((label[j][i]-out),2)
#                     print(out,label[j])
#                     print(self.calculate_oh(label[j][i],out))
                    delta_out = append(delta_out,self.calculate_oh(label[j][i],out))
#                     print(delta_out)
                delta_out = delta_out.reshape(self.output_layer,1)
#                 print(delta_out)
                delta_hid =[]
                for i in range(self.hidden_layer):
                    #print(delta_out.T.shape,output_weight[i].shape)
                    #print(matmul(delta_out.T,output_weight[i].reshape(20,1)))
#                     print(matmul(delta_out.T,output_weight[i].reshape(20,1))*.5*(1-hidden_output[i][0]**2))
                    delta_hid.append(matmul(delta_out.T,output_weight[i].reshape(20,1))*.5*(1-hidden_output[i][0]**2))
                #print(delta_hid)
                delta_hid = array(delta_hid).reshape(self.hidden_layer,1)
#                 print(delta_out.shape,hidden_output.shape)
                output_weight += eta * matmul(hidden_output,delta_out.T)
                x = pattern[j].reshape(1,len(pattern[j]))
                hidden_weight += eta * matmul(delta_hid,x)
                #print(f"y :{hidden_output}")
            count += 1
            print(error)
            if error < emax:
                print(f"cycles completed: {count} \n Hidden layer weights:\n {hidden_weight}\nOutput layer weights: \n{output_weight}\n")
                return


if __name__ == "__main__":
    neuron = MultilayerPerceptron(100,20)    
    hidden = 2*random.random_sample((neuron.hidden_layer,len(X_dataset[0])))-1
    output = 2*random.random_sample((neuron.hidden_layer+1,neuron.output_layer))-1   
    pattern = array([[0,0,-1],[0,1,-1],[1,0,-1],[1,1,-1]],dtype=float)   
    label = array([0,1,1,0])   
    neuron.train(output,hidden,X_dataset,Y_dataset,.01,0.001)
